In [4]:
from pathlib import Path

import cv2, os, pickle

from dex_retargeting.constants import RobotName, RetargetingType, HandType, get_default_config_path
from dex_retargeting.retargeting_config import RetargetingConfig
from dex_retargeting.seq_retarget import SeqRetargeting
from single_hand_detector import SingleHandDetector

import mediapipe as mp
import numpy as np

In [5]:
robot_name = RobotName.ability
retargeting_type = RetargetingType.dexpilot
hand_type = HandType.right

config_path = get_default_config_path(robot_name, retargeting_type, hand_type)
robot_dir = Path().absolute().parent / "ability_hand" / "assets" 
RetargetingConfig.set_default_urdf_dir(str(robot_dir))
retargeting = RetargetingConfig.load_from_file(config_path).build()

video_path = "data/human_hand_video.mp4"
output_path = "data/ability_joints.pkl"

 Mimic joint adaptor enabled. The mimic joint tags in the URDF will be considered during retargeting.
To disable mimic joint adaptor, consider setting ignore_mimic_joint=True in the configuration. 


In [6]:
cap = cv2.VideoCapture(video_path)

detector = SingleHandDetector(hand_type="Right", selfie=False)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

data = []

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    rgb = frame[..., ::-1]
    num_box, joint_pos, keypoint_2d, mediapipe_wrist_rot = detector.detect(rgb)
    
    retargeting_type = retargeting.optimizer.retargeting_type
    indices = retargeting.optimizer.target_link_human_indices
    if retargeting_type == "POSITION":
        indices = indices
        ref_value = joint_pos[indices, :]
    else:
        origin_indices = indices[0, :]
        task_indices = indices[1, :]
        ref_value = joint_pos[task_indices, :] - joint_pos[origin_indices, :]
    qpos = retargeting.retarget(ref_value)
    data.append(qpos)

meta_data = dict(
    config_path=config_path,
    dof=len(retargeting.optimizer.robot.dof_joint_names),
    joint_names=retargeting.optimizer.robot.dof_joint_names,
)

output_path = Path(output_path)
output_path.parent.mkdir(parents=True, exist_ok=True)
with output_path.open("wb") as f:
    pickle.dump(dict(data=data, meta_data=meta_data), f)

retargeting.verbose()
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1722918259.934590  189878 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2 Pro
W0000 00:00:1722918259.944092  312926 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1722918259.950812  312926 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Retargeting 621 times takes: 1.1396910429721174s
Last distance: 0.03218003782018519
